In [5]:
# %%sh
# cd data
# wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet 
# wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet 
# wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-03.parquet 


In [2]:
!python preprocess.py --raw_data_path data --dest_path ./output

In [7]:
!ls output 
# Ans 1 - 4

dv.pkl	test.pkl  train.pkl  valid.pkl


In [11]:
!python train.py

2022/05/30 14:34:42 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/schhabra/miniconda3/envs/py39/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [12]:
# number of parameters logged 17

In [14]:
!python hpo.py

Traceback (most recent call last):
  File "/home/schhabra/experiments/mlops_zoom/m1/hpo.py", line 6, in <module>
    from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
ModuleNotFoundError: No module named 'hyperopt'


In [2]:
from mlflow.tracking import MlflowClient
client = MlflowClient("http://127.0.0.1:5000")

In [3]:
exp = client.get_experiment_by_name('random-forest-hyperopt')
exp

<Experiment: artifact_location='s3://mlflow-artifacts/13', experiment_id='13', lifecycle_stage='active', name='random-forest-hyperopt', tags={}>

In [14]:
run = client.search_runs(experiment_ids=exp.experiment_id , order_by=["metrics.rmse ASC"])

In [16]:
# minimum rmse
sorted([r.data.metrics['rmse'] for r in run])[0]

6.62842574820447

In [20]:
_runs = {r.data.metrics['rmse']:r.info.run_id for r in run}

In [23]:
min(_runs), _runs[min(_runs)]

(6.62842574820447, 'bec2ffe8897347549a90fbedd0e689a3')

In [19]:
_runs

{'bec2ffe8897347549a90fbedd0e689a3': 6.62842574820447,
 'a2ee4e00589d4cfdbab8cdf3e8c29a5f': 6.62972800771013,
 '8ae70b78ce9a4393802755aa870e6e75': 6.62985102203826,
 'dfe71470d51c4b8e8652b906e3c38b56': 6.62985102203826,
 '3458d3bc07a542ff9ef658b574d9c99d': 6.62989321963751,
 'f6512ef90f2f4e0781d2f1f83f3bb0c9': 6.63124192567181,
 '77c1cbf7508e4233afc71010a8f9d36f': 6.63140541341828,
 '29ef677e0ccd4e9c94b4c8572ce8616a': 6.63373002104584,
 '8b4d8dcb0b374b3cadff57aa18b3603a': 6.63374616020161,
 '0bd651f8872a44c2af4710b2a46d196f': 6.63886120702391,
 'ce759fcd92dd4b5aaf760d66093e841d': 6.64213728742921,
 '6cb6baeb99a84921887c9fbbc6d767fc': 6.64511703842987,
 '04a3933098a549b1bb5d84f53b90e190': 6.64703390653905,
 '94655f54b32e40248b0111e17cf6aeb1': 6.6488155653121,
 'd4258544790c4639895b4079fd7daa64': 6.64916050642005,
 'cfeca7de5ef845e69552064e8b6e95d6': 6.65143855937677,
 '57e14127c76d40d8bc77dbeac10e0b4c': 6.65280333206408,
 'fb0db7c7df7445ae92c184eea0416b72': 6.65368383287616,
 '81650bc1d

In [18]:
model_uri = f"runs/{_runs[min(_runs)]}/model"
#mlflow.register_model(model_uri)

In [33]:
!python register_model.py

2022/05/31 09:43:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/schhabra/miniconda3/envs/py39/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
2022/05/31 09:43:28 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: No module named 'boto3'
2022/05/31 09:43:49 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: No module named 'boto3'
2022/05/31 09:44:19 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: No module named 'boto3'
2022/05/31 09:44:47 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: No module named 'boto3'
2022/05/31 09:45:09 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: No module named 'boto3'
best run rmse (6.54788618559542,)
Successfully 

In [25]:
EXPERIMENT_NAME = "random-forest-best-models"
from mlflow.entities import ViewType
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
runs = client.search_runs(
    experiment_ids=experiment.experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"],
    )

In [31]:
runs[0].data.metrics
# abd 6.628

{'valid_rmse': 6.62989321963751, 'test_rmse': 6.54885272240193}

In [32]:
!mlflow --version

mlflow, version 1.26.1
